# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark

In [2]:
findspark.init('/home/i-sip_iot/spark-3.0.1-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('LRprc').getOrCreate()

In [6]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [7]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [55]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [56]:
from pyspark.ml.feature import StringIndexer

indexed = StringIndexer(inputCol='Cruise_line', outputCol='cru')


In [8]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [9]:
from pyspark.ml.linalg import Vector

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
vec = VectorAssembler(inputCols=['Age',
                                 'Tonnage',
                                 'passengers',
                                 'length',
                                 'cabins',
                                 'passenger_density'], outputCol='features')

In [12]:
dff = vec.transform(df)

In [13]:
dff.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|[17.0,101.353,26....|
|    Ecstasy|   Carnival| 22|            70.367|     20.

In [14]:
dff.select('features').show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [15]:
df_LR = dff.select(['features', 'crew'])

In [16]:
df_LR.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [17]:
from pyspark.ml.regression import LinearRegression

In [18]:
LR_Model = LinearRegression(featuresCol='features', labelCol='crew', predictionCol='prediction')

In [44]:
df_train, df_test = df_LR.randomSplit([0.7,0.3])

In [45]:
df_train.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              112|
|   mean|8.265625000000009|
| stddev|3.482162986609807|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [46]:
df_test.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                46|
|   mean| 6.646304347826086|
| stddev|3.3188152897587164|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [47]:
trained_model = LR_Model.fit(df_train)

In [48]:
result = trained_model.evaluate(df_test)

In [49]:
result.r2

0.9338442732164917

In [50]:
result.rootMeanSquaredError

0.8442948607785769

In [51]:
result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-0.38632674656767474|
| 0.35685142551025173|
|-0.41288502590941434|
|   0.610842811413491|
|  1.5704774972333468|
| -0.4761780188117086|
| -0.5316844118653652|
| 0.29720014155546437|
| -0.5812638900670386|
|-0.25829688112653926|
|  0.3410288285526466|
|  -0.294000959265432|
|-0.36886790210810716|
| 0.24839560450551668|
|-0.05635988945246...|
|-0.34436910778208585|
|  -1.350668480305453|
|-0.01701139903447091|
| -0.7580281651147089|
| 0.27318758042338365|
+--------------------+
only showing top 20 rows



In [52]:
unlabeled_data = df_LR.select('features')

In [53]:
final_result = trained_model.transform(unlabeled_data)

In [54]:
final_result.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...|3.9363267465676746|
|[6.0,30.276999999...|3.9363267465676746|
|[26.0,47.262,14.8...|6.3997920012836556|
|[11.0,110.0,29.74...|12.134448095992994|
|[17.0,101.353,26....|10.855485938385732|
|[22.0,70.367,20.5...| 8.656021329294363|
|[15.0,70.367,20.5...| 8.742793245006899|
|[23.0,70.367,20.5...| 8.653931961407926|
|[19.0,70.367,20.5...| 8.693209293171165|
|[6.0,110.23899999...|11.143148574489748|
|[10.0,110.0,29.74...|12.131684411865365|
|[28.0,46.052,14.5...|6.2888307965477805|
|[18.0,70.367,20.5...| 8.705605281130097|
|[17.0,70.367,20.5...| 8.718001269089031|
|[11.0,86.0,21.24,...| 9.629949804579518|
|[8.0,110.0,29.74,...|12.156476387783233|
|[9.0,88.5,21.24,9...|  9.68915718858651|
|[15.0,70.367,20.5...| 8.742793245006899|
|[12.0,88.5,21.24,...|10.448518677400035|
|[20.0,70.367,20.5...| 8.680813305212231|
+--------------------+------------